In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import matplotlib
import tensorflow as tf
import sklearn

2025-05-17 00:31:22.766901: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-17 00:31:22.886877: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-17 00:31:22.983457: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747441883.069154    3367 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747441883.092167    3367 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747441883.269917    3367 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.feature import VectorAssembler
import pandas as pd
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, sum



In [5]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [3]:
import yfinance as yf

In [4]:


ticker = "BA"
data = yf.download(ticker,
                   start="2018-01-01", 
                   end="2020-12-31")    

print(data.head())



YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open   Volume
Ticker              BA          BA          BA          BA       BA
Date                                                               
2018-01-02  282.886444  283.029387  281.514132  281.847685  2978900
2018-01-03  283.801208  284.468315  281.580748  282.028656  3211200
2018-01-04  282.724396  284.392133  281.580793  283.934686  4171700
2018-01-05  294.322266  294.369933  282.771999  282.819637  6177700
2018-01-08  295.570770  296.247387  291.377607  294.150820  4124900


In [5]:
csv_BA = f"{ticker}_BA_historical_train_data_spark.csv"
data.to_csv(csv_BA)

In [6]:
df_BA= pd.read_csv("BA_BA_historical_train_data_spark.csv")

In [7]:
df_BA.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,BA,BA,BA,BA,BA
1,Date,NaN,NaN,NaN,NaN,NaN
2,2018-01-02,282.8864440917969,283.02938722794903,281.5141318186582,281.84768519170586,2978900
3,2018-01-03,283.80120849609375,284.4683150053647,281.58074834802835,282.0286560709894,3211200
4,2018-01-04,282.7243957519531,284.392133204921,281.580792722419,283.93468617650075,4171700


In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

# Parar sessão anterior, se necessário
try:
    spark.stop()
except:
    pass

# SparkSession
spark = SparkSession.builder \
    .appName("PySpark CSV to MySQL") \
    .master("local") \
    .config("spark.jars", "/usr/local/spark-3.4.4-bin-hadoop3/jars/mysql-connector-j-8.0.33.jar") \
    .getOrCreate()

# CSV PATH
file_path = "file:///home/hduser/novo/BA_BA_historical_train_data_spark.csv"

# READING OF THE CSV
df = spark.read.csv(file_path, header=True, inferSchema=True)


df.show(5)


25/05/17 00:33:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+----------+------------------+------------------+------------------+------------------+-------+
|     Price|             Close|              High|               Low|              Open| Volume|
+----------+------------------+------------------+------------------+------------------+-------+
|    Ticker|                BA|                BA|                BA|                BA|     BA|
|      Date|              null|              null|              null|              null|   null|
|2018-01-02| 282.8864440917969|283.02938722794903| 281.5141318186582|281.84768519170586|2978900|
|2018-01-03|283.80120849609375| 284.4683150053647|281.58074834802835| 282.0286560709894|3211200|
|2018-01-04| 282.7243957519531|  284.392133204921|  281.580792722419|283.93468617650075|4171700|
+----------+------------------+------------------+------------------+------------------+-------+
only showing top 5 rows



In [9]:
# print the data type info
df.printSchema()

root
 |-- Price: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- Volume: string (nullable = true)



In [10]:
df.show(5)

+----------+------------------+------------------+------------------+------------------+-------+
|     Price|             Close|              High|               Low|              Open| Volume|
+----------+------------------+------------------+------------------+------------------+-------+
|    Ticker|                BA|                BA|                BA|                BA|     BA|
|      Date|              null|              null|              null|              null|   null|
|2018-01-02| 282.8864440917969|283.02938722794903| 281.5141318186582|281.84768519170586|2978900|
|2018-01-03|283.80120849609375| 284.4683150053647|281.58074834802835| 282.0286560709894|3211200|
|2018-01-04| 282.7243957519531|  284.392133204921|  281.580792722419|283.93468617650075|4171700|
+----------+------------------+------------------+------------------+------------------+-------+
only showing top 5 rows



In [11]:
# Remove the first two lines (CRL and null)
df = df.filter(df.Price != "Ticker").filter(df.Price != "Date")

In [12]:
df = df.toDF("Date", "Close", "High", "Low", "Open", "Volume")

In [13]:


df.show(5)



+----------+------------------+------------------+------------------+------------------+-------+
|      Date|             Close|              High|               Low|              Open| Volume|
+----------+------------------+------------------+------------------+------------------+-------+
|2018-01-02| 282.8864440917969|283.02938722794903| 281.5141318186582|281.84768519170586|2978900|
|2018-01-03|283.80120849609375| 284.4683150053647|281.58074834802835| 282.0286560709894|3211200|
|2018-01-04| 282.7243957519531|  284.392133204921|  281.580792722419|283.93468617650075|4171700|
|2018-01-05|     294.322265625|294.36993271040814|282.77199861116463|282.81963661354325|6177700|
|2018-01-08| 295.5707702636719| 296.2473871438518| 291.3776069209913|294.15082001403056|4124900|
+----------+------------------+------------------+------------------+------------------+-------+
only showing top 5 rows



In [14]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- Volume: string (nullable = true)



In [15]:


# # Count Number of nulls in all Column
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()



+----+-----+----+---+----+------+
|Date|Close|High|Low|Open|Volume|
+----+-----+----+---+----+------+
|   0|    0|   0|  0|   0|     0|
+----+-----+----+---+----+------+



In [16]:
# convert string double
df = df.withColumn("Close", col("Close").cast("double"))

In [17]:
df.printSchema()


root
 |-- Date: string (nullable = true)
 |-- Close: double (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- Volume: string (nullable = true)



In [18]:


df = df.withColumn("High", col("High").cast("double"))
df = df.withColumn("Low", col("Low").cast("double"))
df = df.withColumn("Open", col("Open").cast("double"))
df = df.withColumn("Volume", col("Volume").cast("double"))
df = df.withColumn("Date", col("Date").cast("Date"))



In [19]:


df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Close: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Volume: double (nullable = true)



In [20]:
import mysql.connector
print(mysql.connector.__version__)

9.3.0


In [21]:
df.show(5)

+----------+------------------+------------------+------------------+------------------+---------+
|      Date|             Close|              High|               Low|              Open|   Volume|
+----------+------------------+------------------+------------------+------------------+---------+
|2018-01-02| 282.8864440917969|283.02938722794903| 281.5141318186582|281.84768519170586|2978900.0|
|2018-01-03|283.80120849609375| 284.4683150053647|281.58074834802835| 282.0286560709894|3211200.0|
|2018-01-04| 282.7243957519531|  284.392133204921|  281.580792722419|283.93468617650075|4171700.0|
|2018-01-05|     294.322265625|294.36993271040814|282.77199861116463|282.81963661354325|6177700.0|
|2018-01-08| 295.5707702636719| 296.2473871438518| 291.3776069209913|294.15082001403056|4124900.0|
+----------+------------------+------------------+------------------+------------------+---------+
only showing top 5 rows



In [22]:
# # Count Number of nulls in all Column
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+----+-----+----+---+----+------+
|Date|Close|High|Low|Open|Volume|
+----+-----+----+---+----+------+
|   0|    0|   0|  0|   0|     0|
+----+-----+----+---+----+------+



In [23]:
# MySQL connection
url = "jdbc:mysql://localhost:3306/companies"  # MySQL URL
properties = {
    "user": "root",
    "password": "password",
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [24]:

df_comp_nome = df.withColumn("nome", lit("BA"))

# WRITE IN  MySQL
df_comp_nome.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/companies") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "BA_historico") \
    .option("user", "root") \
    .option("password", "password") \
    .mode("overwrite") \
    .save()


In [25]:
# # Count Number of nulls in all Column
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+----+-----+----+---+----+------+
|Date|Close|High|Low|Open|Volume|
+----+-----+----+---+----+------+
|   0|    0|   0|  0|   0|     0|
+----+-----+----+---+----+------+



mySQL